# Photometric Redshift Estimation

In this notebook, we will show how we can estimate the redshifts of galaxies using some machine learning methods. For this to be done, we will use data engineering techniques and thus be able to use it.

## Libraries

In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import KFold,ShuffleSplit
# Neural Network Libs

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from sklearn.metrics import confusion_matrix
import keras as ks
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import layers
from tensorflow.keras import regularizers

## Functions

In [2]:
def rmse_ann(y_true, y_pred):
    diff = keras.backend.square(
        (y_pred - y_true))
    return keras.backend.sqrt(keras.backend.mean(diff))
def get_features_targets2(data):
    '''Extract the colors using the mag_derred.
    Here I'm using the DES-mag, for other survey a change is needed'''
    features = np.zeros(shape=(len(data), 5))

    features[:, 0] = data['MAG_AUTO_G_DERED'].values - \
        data['MAG_AUTO_R_DERED'].values
    features[:, 1] = data['MAG_AUTO_R_DERED'].values - \
        data['MAG_AUTO_I_DERED'].values
    features[:, 2] = data['MAG_AUTO_I_DERED'].values - \
        data['MAG_AUTO_Z_DERED'].values
    features[:, 3] = data['MAG_AUTO_Z_DERED'].values - \
        data['MAG_AUTO_Y_DERED'].values
    #features[:, 4] = data['WAVG_MAG_PSF_G_DERED'].values - \
    #   data['WAVG_MAG_PSF_R_DERED'].values
    #features[:, 5] = data['WAVG_MAG_PSF_R_DERED'].values - \
    #    data['WAVG_MAG_PSF_I_DERED'].values
    #features[:, 6] = data['WAVG_MAG_PSF_I_DERED'].values - \
    #   data['WAVG_MAG_PSF_Z_DERED'].values
    #features[:, 7] = data['WAVG_MAG_PSF_Z_DERED'].values - \
    #  data['WAVG_MAG_PSF_Y_DERED'].values

    features[:, 4] = data["MAG_AUTO_I_DERED"].values
    #features[:, 9] = data["WAVG_MAG_PSF_I_DERED"].values

    targets = data['z'].values
    return features, targets
def tts_split(X, y, size, splits):
    '''Split the data in Train and
     test using the Shuffle split'''

    rs = ShuffleSplit(n_splits=splits, test_size=size)

    rs.get_n_splits(X)

    for train_index, test_index in rs.split(X, y):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return X_train, X_test, y_train, y_test

## Load data

In [3]:
data = Table.read("../input/vipers.fits").to_pandas()

## Preprocessing Data

In [4]:
feat = ['MAG_AUTO_G','MAG_AUTO_R','MAG_AUTO_I','MAG_AUTO_Z','MAG_AUTO_Y',
        'MAG_AUTO_G_DERED','MAG_AUTO_R_DERED','MAG_AUTO_I_DERED','MAG_AUTO_Z_DERED','MAG_AUTO_Y_DERED',
        "WAVG_MAG_PSF_G","WAVG_MAG_PSF_R","WAVG_MAG_PSF_I","WAVG_MAG_PSF_Z","WAVG_MAG_PSF_Y"
       ,'WAVG_MAG_PSF_G_DERED','WAVG_MAG_PSF_R_DERED','WAVG_MAG_PSF_I_DERED','WAVG_MAG_PSF_Z_DERED','WAVG_MAG_PSF_Y_DERED']


In [5]:
data.loc[data[feat[0]]==99,feat[0]] = data[data[feat[0]]!=99][feat[0]].max()
data.loc[data[feat[1]]==99,feat[1]] = data[data[feat[1]]!=99][feat[1]].max()
data.loc[data[feat[2]]==99,feat[2]] = data[data[feat[2]]!=99][feat[2]].max()
data.loc[data[feat[3]]==99,feat[3]] = data[data[feat[3]]!=99][feat[3]].max()
data.loc[data[feat[4]]==99,feat[4]] = data[data[feat[4]]!=99][feat[4]].max()
data.loc[data[feat[5]]>90,feat[5]] = data[data[feat[5]]<90][feat[5]].max()
data.loc[data[feat[6]]>90,feat[6]] = data[data[feat[6]]<90][feat[6]].max()
data.loc[data[feat[7]]>90,feat[7]] = data[data[feat[7]]<90][feat[7]].max()
data.loc[data[feat[8]]>90,feat[8]] = data[data[feat[8]]<90][feat[8]].max()
data.loc[data[feat[9]]>90,feat[9]] = data[data[feat[9]]<90][feat[9]].max()
data.loc[data[feat[10]]>90,feat[10]] = data[data[feat[10]]<90][feat[10]].max()
data.loc[data[feat[11]]>90,feat[11]] = data[data[feat[11]]<90][feat[11]].max()
data.loc[data[feat[12]]>90,feat[12]] = data[data[feat[12]]<90][feat[12]].max()
data.loc[data[feat[13]]>90,feat[13]] = data[data[feat[13]]<90][feat[13]].max()
data.loc[data[feat[14]]>90,feat[14]] = data[data[feat[14]]<90][feat[14]].max()
data.loc[data[feat[15]]>90,feat[15]] = data[data[feat[15]]<90][feat[15]].max()
data.loc[data[feat[16]]>90,feat[16]] = data[data[feat[16]]<90][feat[16]].max()
data.loc[data[feat[17]]>90,feat[17]] = data[data[feat[17]]<90][feat[17]].max()
data.loc[data[feat[18]]>90,feat[18]] = data[data[feat[18]]<90][feat[18]].max()
data.loc[data[feat[19]]>90,feat[19]] = data[data[feat[19]]<90][feat[19]].max()

In [6]:
X,y = get_features_targets2(data)
y = y.reshape(-1,1)

In [7]:
X.shape

(47658, 5)

### Discretize

Here discretize data to obatain also the PDF's

In [8]:
from sklearn.preprocessing import KBinsDiscretizer
kbins = KBinsDiscretizer(200,encode = "onehot",strategy = "uniform")
kbins.fit(y.reshape(-1,1))
y_bins = kbins.transform(y.reshape(-1,1))


In [9]:
from scipy.sparse import hstack,vstack

In [10]:
y_total = hstack([y_bins,y])
y_total.shape

(47658, 201)

In [11]:
y_total = y_total.toarray()

In [12]:
# concatenate the mag for plot purpose only

X = np.concatenate((X,data[['MAG_AUTO_G_DERED','MAG_AUTO_R_DERED','MAG_AUTO_I_DERED','MAG_AUTO_Z_DERED','MAG_AUTO_Y_DERED',]].values),axis = 1 )

In [13]:
y_total.shape

(47658, 201)

In [14]:
X.shape

(47658, 10)

## Model Training

In [15]:
n_inputs = X.shape[1]
n_inputs

10

### ANN Model

In [16]:
EarlyStop = EarlyStopping(monitor='reg_mse', mode='min', patience=20)
BATCH_SIZE = 64
STEPS_PER_EPOCH = len(data)//BATCH_SIZE
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        0.0001,
        decay_steps=STEPS_PER_EPOCH*1000,
        decay_rate=1,
        staircase=False)

In [17]:
inputs = keras.layers.Input(5)
x = BatchNormalization()(inputs)
x = Dense(20, kernel_initializer='normal',  kernel_constraint=max_norm(2.) ,activation='tanh',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                              bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)) (x)
x = BatchNormalization()(x)
x = Dense(15, kernel_initializer='normal',  kernel_constraint=max_norm(2.) ,activation='tanh',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                              bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)) (x)
x = BatchNormalization()(x)
x = Dense(10, kernel_initializer='normal',  kernel_constraint=max_norm(2.) ,activation='elu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                              bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)) (x)
output1 = Dense(1,activation = "linear",name = "reg") (x)
output2 = Dense(200,activation = "softmax",name ="pdf")(x)
model = keras.Model(inputs=inputs, outputs=[output1,output2], name="ann-photoz")

In [18]:
model.summary()

Model: "ann-photoz"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 5)            20          input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 20)           120         batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 20)           80          dense[0][0]                      
_________________________________________________________________________________________

In [19]:
#opt = ks.optimizers.Adamax(lr_schedule)
opt = ks.optimizers.RMSprop(lr_schedule)
model.compile(
    loss={'reg': 'mean_absolute_error', 
                    'pdf': keras.losses.CategoricalCrossentropy()},loss_weights=[0.1,0.9],
              optimizer=opt,
              
    metrics={'pdf': "acc",
                      'reg': "mse"})

In [20]:
X_train,X_test,y_train,y_test = tts_split(X,y_total,0.3,5)

In [ ]:
folds = KFold(n_splits=10, shuffle=True, random_state=None)
models = []
pred = np.zeros(3336)
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    fold_ += 1
    print("Fold {}".format(fold_))
    X_t,y_t = X_train[trn_idx], y_train[trn_idx]
    X_test,y_test = X_train[val_idx],y_train[val_idx]
    # shape
    # trainning
    history = model.fit(X_t[:,:5], {'pdf': y_t[:,:200], 'reg': y_t[:,200]}, 
                    batch_size = 64,epochs= 256 ,validation_split = 0.2, callbacks=[EarlyStop],verbose = 0)
    model_name = "ann-"+str(fold_)
    model.save("../models/"+model_name)
    predictions = model.predict(X_test[:,:5])
    rmse = np.sqrt(mean_squared_error(y_test[:,200],predictions[0]))
    models.append({"model":model_name, "rmse": rmse})
    pred += predictions[0].flatten()/ folds.n_splits

Fold 1
INFO:tensorflow:Assets written to: ../models/ann-1/assets
Fold 2
INFO:tensorflow:Assets written to: ../models/ann-2/assets
Fold 3
INFO:tensorflow:Assets written to: ../models/ann-3/assets
Fold 4
INFO:tensorflow:Assets written to: ../models/ann-4/assets
Fold 5
INFO:tensorflow:Assets written to: ../models/ann-5/assets
Fold 6
INFO:tensorflow:Assets written to: ../models/ann-6/assets
Fold 7
INFO:tensorflow:Assets written to: ../models/ann-7/assets
Fold 8


In [ ]:
models

In [ ]:
reconstructed_model = keras.models.load_model("../models/ann-10") # choose the minor value of rmse
test_predictions = reconstructed_model.predict(X_test[:,:5])


print("Testing set Mean Abs Error: {:5.4f} ".format(mean_absolute_error(y_test[:,200],test_predictions[0])))
print("\n")
print("Testing set Mean Square Error: {:5.4f} ".format(mean_squared_error(y_test[:,200],test_predictions[0])))
print("\n")
print("Testing set Root Mean Square Error: {:5.4f} ".format(np.sqrt(mean_squared_error(y_test[:,200],test_predictions[0]))))

In [ ]:
zspec = y_test[:,200].flatten()
zphot = test_predictions[0].flatten()
pdf = test_predictions[1]
x_plot = np.linspace(0,3.5,200)

In [ ]:
plt.figure(figsize=(16, 8))
plt.hist2d(zspec,zphot, bins= 100,density=True,cmap = "plasma")
plt.colorbar()
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.grid()
#plt.show()
#plt.savefig("plots/ann_hist2d.png")
##plt.close()

In [ ]:
plt.figure(figsize=(16, 8))
plt.scatter(zspec, zphot, s=1, c="k")
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.grid()
_ = plt.plot([-100, 100], [-100, 100],color = "red")
#plt.show()
#plt.savefig("plots/scatter_ann.png",dpi = 300)
#plt.close()


In [ ]:
from scipy.stats import gaussian_kde
plt.figure(figsize=(16,8))
plt.title("Probabilty using KDE- Estimation")
xy = np.hstack([zspec.reshape(-1,1),zphot.reshape(-1,1)]).T
z = gaussian_kde(xy)(xy)
plt.scatter(zspec,zphot,c=z,s=5,cmap = "viridis")
plt.xlabel("True Redshift")
plt.ylabel("Predicted Redshift")
plt.grid()
plt.colorbar()
#plt.savefig("plots/scatter_probs_ann_rafael.png",dpi =300)

In [ ]:
plt.figure(figsize=(16,8))
plt.scatter(zspec,zphot,s=5,cmap = "viridis")
plt.xlabel("True Redshift")
plt.ylabel("Predicted Redshift")
plt.grid()
#plt.savefig("plots/scatter.ann_rafael.png",dpi =300)

In [ ]:
redshift = pd.DataFrame()
redshift["z_phot"] = zphot
redshift["z_spec"] = zspec

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(redshift[(redshift["z_phot"] > 1.) & (redshift["z_phot"] < 1.5)]["z_spec"].values,label = "1.<z<1.5",bins = 100, color = "b")
plt.hist(redshift[(redshift["z_phot"] > 0.9) & (redshift["z_phot"] < 1.)]["z_spec"].values,label = "0.9<z<1.0",bins = 100,color = "r")
plt.hist(redshift[(redshift["z_phot"] > 0.8) & (redshift["z_phot"] < 0.9)]["z_spec"].values,label = "0.8<z<0.9",bins = 100,color = "g")
plt.hist(redshift[(redshift["z_phot"] > 0.7) & (redshift["z_phot"] < 0.8)]["z_spec"].values,label = "0.7<z<0.8",bins = 100,color = "yellow")
plt.hist(redshift[(redshift["z_phot"] > 0.6) & (redshift["z_phot"] < 0.7)]["z_spec"].values,label = "0.6<z<0.7",bins = 100,color = "brown")
plt.hist(redshift[(redshift["z_phot"] > 0.5) & (redshift["z_phot"] < 0.6)]["z_spec"].values,label = "0.5<z<0.6",bins = 100,color = "pink")
plt.hist(redshift[redshift["z_phot"] < 0.5]["z_spec"].values,label = "z<0.5",bins = 100)
plt.xlabel("Redshift spectoscopic")
plt.legend()

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(redshift[redshift["z_phot"] < 0.5]["z_spec"].values,label = "z<0.5",bins = 100)
plt.hist(redshift[(redshift["z_phot"] > 0.5) & (redshift["z_phot"] < 0.6)]["z_spec"].values,label = "0.5<z<0.6",bins = 100,color = "pink" )
plt.hist(redshift[(redshift["z_phot"] > 0.6) & (redshift["z_phot"] < 0.7)]["z_spec"].values,label = "0.6<z<0.7",bins = 100,color = "brown")
plt.hist(redshift[(redshift["z_phot"] > 0.7) & (redshift["z_phot"] < 0.8)]["z_spec"].values,label = "0.7<z<0.8",bins = 100,color = "yellow")
plt.hist(redshift[(redshift["z_phot"] > 0.8) & (redshift["z_phot"] < 0.9)]["z_spec"].values,label = "0.8<z<0.9",bins = 100,color = "g")
plt.hist(redshift[(redshift["z_phot"] > 0.9) & (redshift["z_phot"] < 1.)]["z_spec"].values,label = "0.9<z<1.0",bins = 100,color = "r")
plt.hist(redshift[(redshift["z_phot"] > 1.) & (redshift["z_phot"] < 1.5)]["z_spec"].values,label = "1.<z<1.5",bins = 100,color = "b")
plt.xlabel("Redshift spectoscopic")
plt.legend()